# Laboratorio No. 7

## Ejercicio No. 1

#### a) Formulación del modelo de transporte

Variables:
- Hay 3 centros de distribución y 5 concesionarios.
- x_ij: cantidad de automóviles enviados desde el centro de distribución i al concesionario j.
- c_ij: costo de envío desde el centro i al concesionario j, que depende de la distancia.
- s_i: oferta de automóviles en el centro i.
- d_j: demanda de automóviles del concesionario j.

El costo de transporte se calcula multiplicando la distancia por milla entre i y j por $25 y el número de camiones necesarios para cumplir con la demanda, donde cada camión transporta 18 automóviles.

**Función objetivo**:
Minimizar el costo total de transporte, que es la suma de x_ij x c_ij para todos los i y j.

**Restricciones**:
1. La cantidad total enviada desde cada centro i no puede exceder la oferta s_i.
2. La cantidad total recibida por cada concesionario j debe satisfacer su demanda d_j.


### b) Resolver el problema usando la librer ́ıa JuMP. Indique cu ́al es la distribuci ́on  ́optima, y el costo asociado.

In [1]:
using Pkg
Pkg.add("JuMP")
Pkg.add("GLPK")

using JuMP, GLPK

    Updating registry at `C:\Users\bcarr\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\bcarr\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\bcarr\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\bcarr\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\bcarr\.julia\environments\v1.10\Manifest.toml`


In [105]:
oferta = [400, 200, 150]  # Oferta de cada centro
oferta = [ceil(value/18) for value in oferta]  # Convertir a camiones
demanda = [100, 200, 150, 160, 140]  # Demanda de cada concesionario
demanda = [ceil(value/18) for value in demanda]  # Convertir a camiones

distancias = [
    [100, 150, 200, 140, 35],  # Distancias desde centro 1
    [50, 70, 60, 65, 80],      # Distancias desde centro 2
    [40, 90, 100, 150, 130]    # Distancias desde centro 3
]

3-element Vector{Vector{Int64}}:
 [100, 150, 200, 140, 35]
 [50, 70, 60, 65, 80]
 [40, 90, 100, 150, 130]

In [106]:
# Convertimos las distancias a costos multiplicándolas por 25 (costo por milla)
costo = [d .* 25 for d in distancias]

3-element Vector{Vector{Int64}}:
 [2500, 3750, 5000, 3500, 875]
 [1250, 1750, 1500, 1625, 2000]
 [1000, 2250, 2500, 3750, 3250]

In [107]:
# Creamos el modelo de optimización
model = Model(GLPK.Optimizer)

A JuMP Model
├ solver: GLPK
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [108]:
# Definimos las variables de decisión: cantidad enviada de cada centro a cada concesionario
@variable(model, x[1:3, 1:5] >= 0)  # x[i,j] es la cantidad enviada desde centro i a concesionario j

# Definimos la función objetivo: minimizar el costo total de transporte
@objective(model, Min, sum(x[i,j] * costo[i][j] for i in 1:3, j in 1:5))

2500 x[1,1] + 3750 x[1,2] + 5000 x[1,3] + 3500 x[1,4] + 875 x[1,5] + 1250 x[2,1] + 1750 x[2,2] + 1500 x[2,3] + 1625 x[2,4] + 2000 x[2,5] + 1000 x[3,1] + 2250 x[3,2] + 2500 x[3,3] + 3750 x[3,4] + 3250 x[3,5]

In [109]:
# Restricciones de oferta: no se puede enviar más de lo disponible en cada centro
for i in 1:3
    @constraint(model, sum(x[i,j] for j in 1:5) <= oferta[i])
end

# Restricciones de demanda: cada concesionario debe recibir lo que necesita
for j in 1:5
    @constraint(model, sum(x[i,j] for i in 1:3) == demanda[j])
end

In [110]:
# Resolución del problema
optimize!(model)

println("Distribución óptima de automóviles:")
for i in 1:3
    for j in 1:5
        println("Desde centro $i a concesionario $j: ", value(x[i,j]))
    end
end

println("Costo total: ", objective_value(model))

Distribución óptima de automóviles:
Desde centro 1 a concesionario 1: 0.0
Desde centro 1 a concesionario 2: 6.0
Desde centro 1 a concesionario 3: 0.0
Desde centro 1 a concesionario 4: 9.0
Desde centro 1 a concesionario 5: 8.0
Desde centro 2 a concesionario 1: 0.0
Desde centro 2 a concesionario 2: 3.0
Desde centro 2 a concesionario 3: 9.0
Desde centro 2 a concesionario 4: 0.0
Desde centro 2 a concesionario 5: 0.0
Desde centro 3 a concesionario 1: 6.0
Desde centro 3 a concesionario 2: 3.0
Desde centro 3 a concesionario 3: 0.0
Desde centro 3 a concesionario 4: 0.0
Desde centro 3 a concesionario 5: 0.0
Costo total: 92500.0


## Ejercicio No. 2

#### (a) Asuma que en el problema anterior, el Concesionario 5 requiere 200 autom ́oviles. Formular y resolver el nuevo problema de transporte, comparando la soluci ́on con la del Problema 1.

El Concesionario 5 aumenta su demanda de 140 a 200 automóviles, mientras que la oferta de los centros de distribución permanece igual. El objetivo sigue siendo minimizar el costo total de transporte de los automóviles desde tres centros de distribución a cinco concesionarios, considerando el costo de 25 dólares por milla recorrida. Las variables de decisión x_ij representan la cantidad de automóviles enviados desde el centro i al concesionario j. El modelo está sujeto a restricciones de oferta, que aseguran que los centros de distribución no envíen más de lo que pueden producir, y restricciones de demanda, que requieren que cada concesionario reciba exactamente la cantidad de automóviles que necesita.

In [111]:

oferta = [400, 200, 150]  # Oferta de cada centro
oferta = [ceil(value/18) for value in oferta]  # Convertir a camiones
demanda = [100, 200, 150, 160, 200]  # Nueva demanda de cada concesionario
demanda = [ceil(value/18) for value in demanda]  # Convertir a camiones

distancias = [
    [100, 150, 200, 140, 35],  # Distancias desde centro 1
    [50, 70, 60, 65, 80],      # Distancias desde centro 2
    [40, 90, 100, 150, 130]    # Distancias desde centro 3
]

3-element Vector{Vector{Int64}}:
 [100, 150, 200, 140, 35]
 [50, 70, 60, 65, 80]
 [40, 90, 100, 150, 130]

In [112]:
# Convertimos las distancias a costos multiplicándolas por 25 (costo por milla)
costo = [d .* 25 for d in distancias]

# Creamos el modelo de optimización
model = Model(GLPK.Optimizer)

A JuMP Model
├ solver: GLPK
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [113]:
# Definimos las variables de decisión: cantidad enviada de cada centro a cada concesionario
@variable(model, x[1:3, 1:5] >= 0)  # x[i,j] es la cantidad enviada desde centro i a concesionario j

# Definimos la función objetivo: minimizar el costo total de transporte
@objective(model, Min, sum(x[i,j] * ceil(costo[i][j]) for i in 1:3, j in 1:5))


2500 x[1,1] + 3750 x[1,2] + 5000 x[1,3] + 3500 x[1,4] + 875 x[1,5] + 1250 x[2,1] + 1750 x[2,2] + 1500 x[2,3] + 1625 x[2,4] + 2000 x[2,5] + 1000 x[3,1] + 2250 x[3,2] + 2500 x[3,3] + 3750 x[3,4] + 3250 x[3,5]

In [114]:
# Restricciones de oferta: no se puede enviar más de lo disponible en cada centro
for i in 1:3
    @constraint(model, sum(x[i,j] for j in 1:5) <= oferta[i])
end

# Restricciones de demanda: cada concesionario debe recibir exactamente su demanda
for j in 1:5
    @constraint(model, sum(x[i,j] for i in 1:3) == demanda[j])
end

In [115]:
# Resolución del problema
optimize!(model)

# Mostramos la solución
println("Distribución óptima de automóviles:")
for i in 1:3
    for j in 1:5
        println("Desde centro $i a concesionario $j: ", value(x[i,j]))
    end
end

# Mostramos el costo total
println("Costo total: ", objective_value(model))

Distribución óptima de automóviles:
Desde centro 1 a concesionario 1: 6.0
Desde centro 1 a concesionario 2: 0.0
Desde centro 1 a concesionario 3: 0.0
Desde centro 1 a concesionario 4: 9.0
Desde centro 1 a concesionario 5: 12.0
Desde centro 2 a concesionario 1: 0.0
Desde centro 2 a concesionario 2: 0.0
Desde centro 2 a concesionario 3: 12.0
Desde centro 2 a concesionario 4: 0.0
Desde centro 2 a concesionario 5: 0.0
Desde centro 3 a concesionario 1: 0.0
Desde centro 3 a concesionario 2: 12.0
Desde centro 3 a concesionario 3: -3.0
Desde centro 3 a concesionario 4: 0.0
Desde centro 3 a concesionario 5: 0.0
Costo total: 94500.0


Comparación:
- Demanda no cubierta:
En la primera solución, toda la demanda fue cubierta correctamente sin ningún déficit, ya que no hay valores negativos ni inconsistencias en la distribución.
En la segunda solución, el valor de -3 automóviles asignado al Concesionario 3 indica que no se pudo cubrir la totalidad de la demanda para ese concesionario, lo que generó un déficit. Esto sugiere que, mientras algunos concesionarios (como el Concesionario 5) recibieron más automóviles, otros concesionarios no recibieron lo suficiente, lo que afectó la distribución.
- Distribución:
En la primera solución, el Concesionario 1 recibió 6 automóviles desde el Centro 3, mientras que en la segunda solución, los automóviles provienen del Centro 1.
El Concesionario 2 recibió 6 automóviles en total (distribuidos entre los Centros 1 y 3) en la primera solución, mientras que en la segunda solución recibió 12 automóviles, todos provenientes del Centro 3.
El Concesionario 3, en la primera solución, recibió 9 automóviles del Centro 2 sin problemas. En la segunda solución, aunque se intentó asignar 12 automóviles, no se pudo satisfacer la demanda completa y surgió un déficit de 3 automóviles.
El Concesionario 5 en la segunda solución recibió más automóviles (12 en comparación con los 8 de la primera solución), lo que refleja que se priorizó cubrir la demanda del Concesionario 5 en detrimento del Concesionario 3.
- Costo total:
Costo en la primera solución: $92,500.
Costo en la segunda solución: $94,500.
El costo total es más alto en la segunda solución, principalmente debido a que más automóviles fueron enviados, pero también refleja la desventaja de no cumplir completamente la demanda del Concesionario 3. Aunque el Concesionario 5 recibió más automóviles.


#### (b) Asuma que en el problema anterior, el Centro 3 produce 200 autom ́oviles. Formular y resolver el nuevo problema de transporte, comparando la soluci ́on con la del Problema 1.


El aumento de la producción en el Centro 3 se basa en minimizar el costo total de envío de automóviles desde tres centros de distribución a cinco concesionarios, donde cada ruta tiene un costo asociado basado en la distancia en millas multiplicada por 25 dólares por milla recorrida. La cantidad de automóviles enviados desde cada centro a cada concesionario se modela mediante variables de decisión x_ij, que representan los automóviles enviados desde el centro i al concesionario j. El objetivo es minimizar el costo total, sujeto a las restricciones de oferta, que garantizan que la cantidad enviada desde cada centro no exceda su capacidad, y las restricciones de demanda, que aseguran que cada concesionario reciba exactamente el número de automóviles que requiere. Con el aumento en la oferta del Centro 3 de 150 a 200 automóviles, el modelo busca una distribución óptima que minimice los costos mientras satisface completamente la demanda de los concesionarios.

In [117]:
oferta = [400, 200, 200]  # Centro 3 ahora produce 200 automóviles
oferta = [ceil(value/18) for value in oferta]  # Convertir a camiones
demanda = [100, 200, 150, 160, 140]  # Demanda de cada concesionario (sin cambios)
demanda = [ceil(value/18) for value in demanda]  # Convertir a camiones

distancias = [
    [100, 150, 200, 140, 35],  # Distancias desde centro 1
    [50, 70, 60, 65, 80],      # Distancias desde centro 2
    [40, 90, 100, 150, 130]    # Distancias desde centro 3
]

3-element Vector{Vector{Int64}}:
 [100, 150, 200, 140, 35]
 [50, 70, 60, 65, 80]
 [40, 90, 100, 150, 130]

In [118]:
# Convertimos las distancias a costos multiplicándolas por 25 (costo por milla)
costo = [d .* 25 for d in distancias]

# Creamos el modelo de optimización
model = Model(GLPK.Optimizer)

A JuMP Model
├ solver: GLPK
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [119]:
# Definimos las variables de decisión: cantidad enviada de cada centro a cada concesionario
@variable(model, x[1:3, 1:5] >= 0)  # x[i,j] es la cantidad enviada desde centro i a concesionario j

# Definimos la función objetivo: minimizar el costo total de transporte
@objective(model, Min, sum(x[i,j] * ceil(costo[i][j]) for i in 1:3, j in 1:5))

2500 x[1,1] + 3750 x[1,2] + 5000 x[1,3] + 3500 x[1,4] + 875 x[1,5] + 1250 x[2,1] + 1750 x[2,2] + 1500 x[2,3] + 1625 x[2,4] + 2000 x[2,5] + 1000 x[3,1] + 2250 x[3,2] + 2500 x[3,3] + 3750 x[3,4] + 3250 x[3,5]

In [120]:
# Restricciones de oferta: no se puede enviar más de lo disponible en cada centro
for i in 1:3
    @constraint(model, sum(x[i,j] for j in 1:5) <= oferta[i])
end

# Restricciones de demanda: cada concesionario debe recibir exactamente su demanda
for j in 1:5
    @constraint(model, sum(x[i,j] for i in 1:3) == demanda[j])
end

In [121]:
# Resolución del problema
optimize!(model)

# Mostramos la solución
println("Distribución óptima de automóviles:")
for i in 1:3
    for j in 1:5
        println("Desde centro $i a concesionario $j: ", value(x[i,j]))
    end
end

# Mostramos el costo total
println("Costo total: ", objective_value(model))

Distribución óptima de automóviles:
Desde centro 1 a concesionario 1: 0.0
Desde centro 1 a concesionario 2: 3.0
Desde centro 1 a concesionario 3: 0.0
Desde centro 1 a concesionario 4: 9.0
Desde centro 1 a concesionario 5: 8.0
Desde centro 2 a concesionario 1: 0.0
Desde centro 2 a concesionario 2: 3.0
Desde centro 2 a concesionario 3: 9.0
Desde centro 2 a concesionario 4: 0.0
Desde centro 2 a concesionario 5: 0.0
Desde centro 3 a concesionario 1: 6.0
Desde centro 3 a concesionario 2: 6.0
Desde centro 3 a concesionario 3: 0.0
Desde centro 3 a concesionario 4: 0.0
Desde centro 3 a concesionario 5: 0.0
Costo total: 88000.0


Comparación:
- Demanda no cubierta:
En ambas soluciones, toda la demanda ha sido cubierta adecuadamente, ya que no hay déficits en la asignación de automóviles. Por lo tanto, en la primera y en la tercera solución, la distribución de los automóviles cumplió con los requisitos de los concesionarios.
- Distribución:
En ambas soluciones, los Concesionarios 1, 3, 4 y 5 reciben la misma cantidad de automóviles desde los mismos centros.
El Concesionario 2:
En la primera solución, recibe 6 automóviles en total, distribuidos como 6 del Centro 1 y 3 del Centro 2.
En la tercera solución, recibe 9 automóviles en total, distribuidos como 3 del Centro 1, 3 del Centro 2 y 6 del Centro 3. Esto refleja una mayor participación del Centro 3 en la tercera solución, lo que permitió reducir el envío del Centro 1.
Este cambio en la distribución muestra que la tercera solución optimiza mejor la asignación de automóviles hacia el Concesionario 2, logrando un equilibrio más eficiente en los envíos.
- Costo total:
Costo en la primera solución: $92,500.
Costo en la tercera solución: $88,000.
El costo en la tercera solucion es menor, con una reducción de $4,500 en el costo total. Esta reducción se debe principalmente a la mayor utilización del Centro 3 para el Concesionario 2, lo que disminuyó los costos asociados con el Centro 1 y mejoró la eficiencia de las rutas de transporte.

## Ejercicio No. 3

In [1]:
import Pkg
Pkg.add("JuMP")
Pkg.add("GLPK")


    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [2]:
using JuMP, GLPK


### Problema 1

In [3]:
# Matriz de costos del problema (i)
cost = [
    3 8 2 10 3;
    6 5 2 7 5;
    6 4 2 7 5;
    8 4 2 3 5;
    7 8 6 7 7
]


5×5 Matrix{Int64}:
 3  8  2  10  3
 6  5  2   7  5
 6  4  2   7  5
 8  4  2   3  5
 7  8  6   7  7

In [4]:
# Definir el problema de optimización
model = Model(GLPK.Optimizer)

# Número de trabajadores y tareas
n_workers = size(cost, 1)
n_tasks = size(cost, 2)

# Variables de decisión: x[i,j] = 1 si el trabajador i hace la tarea j, 0 de lo contrario
@variable(model, x[1:n_workers, 1:n_tasks], Bin)


5×5 Matrix{VariableRef}:
 x[1,1]  x[1,2]  x[1,3]  x[1,4]  x[1,5]
 x[2,1]  x[2,2]  x[2,3]  x[2,4]  x[2,5]
 x[3,1]  x[3,2]  x[3,3]  x[3,4]  x[3,5]
 x[4,1]  x[4,2]  x[4,3]  x[4,4]  x[4,5]
 x[5,1]  x[5,2]  x[5,3]  x[5,4]  x[5,5]

In [5]:
# Función objetivo: minimizar el costo total de asignación
@objective(model, Min, sum(cost[i, j] * x[i, j] for i in 1:n_workers for j in 1:n_tasks))


3 x[1,1] + 8 x[1,2] + 2 x[1,3] + 10 x[1,4] + 3 x[1,5] + 6 x[2,1] + 5 x[2,2] + 2 x[2,3] + 7 x[2,4] + 5 x[2,5] + 6 x[3,1] + 4 x[3,2] + 2 x[3,3] + 7 x[3,4] + 5 x[3,5] + 8 x[4,1] + 4 x[4,2] + 2 x[4,3] + 3 x[4,4] + 5 x[4,5] + 7 x[5,1] + 8 x[5,2] + 6 x[5,3] + 7 x[5,4] + 7 x[5,5]

In [6]:
# Restricción: cada trabajador hace una tarea
@constraint(model, [i in 1:n_workers], sum(x[i, j] for j in 1:n_tasks) == 1)

# Restricción: cada tarea la hace solo un trabajador
@constraint(model, [j in 1:n_tasks], sum(x[i, j] for i in 1:n_workers) == 1)


5-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 x[1,1] + x[2,1] + x[3,1] + x[4,1] + x[5,1] = 1
 x[1,2] + x[2,2] + x[3,2] + x[4,2] + x[5,2] = 1
 x[1,3] + x[2,3] + x[3,3] + x[4,3] + x[5,3] = 1
 x[1,4] + x[2,4] + x[3,4] + x[4,4] + x[5,4] = 1
 x[1,5] + x[2,5] + x[3,5] + x[4,5] + x[5,5] = 1

In [7]:
# Resolver el modelo
optimize!(model)

# Mostrar resultados
println("Estado del modelo: ", termination_status(model))
println("Costo total: ", objective_value(model))

# Mostrar la asignación de cada trabajador
for i in 1:n_workers, j in 1:n_tasks
    if value(x[i, j]) > 0.5
        println("Trabajador ", i, " hace la tarea ", j)
    end
end


Estado del modelo: OPTIMAL
Costo total: 19.0
Trabajador 1 hace la tarea 1
Trabajador 2 hace la tarea 3
Trabajador 3 hace la tarea 2
Trabajador 4 hace la tarea 4
Trabajador 5 hace la tarea 5


### Problema 2

In [9]:
# Matriz de costos del problema (ii)
cost2 = [
    3 9 2 2 7;
    6 1 5 6 6;
    9 4 7 10 3;
    2 5 4 2 1;
    9 6 2 4 6
]


5×5 Matrix{Int64}:
 3  9  2   2  7
 6  1  5   6  6
 9  4  7  10  3
 2  5  4   2  1
 9  6  2   4  6

In [11]:
# Definir el problema de optimización
model2 = Model(GLPK.Optimizer)

# Número de trabajadores y tareas
n_workers2 = size(cost2, 1)
n_tasks2 = size(cost2, 2)

# Variables de decisión: x[i,j] = 1 si el trabajador i hace la tarea j, 0 de lo contrario
@variable(model2, x[1:n_workers2, 1:n_tasks2], Bin)


5×5 Matrix{VariableRef}:
 x[1,1]  x[1,2]  x[1,3]  x[1,4]  x[1,5]
 x[2,1]  x[2,2]  x[2,3]  x[2,4]  x[2,5]
 x[3,1]  x[3,2]  x[3,3]  x[3,4]  x[3,5]
 x[4,1]  x[4,2]  x[4,3]  x[4,4]  x[4,5]
 x[5,1]  x[5,2]  x[5,3]  x[5,4]  x[5,5]

In [12]:
# Función objetivo: minimizar el costo total de asignación
@objective(model2, Min, sum(cost2[i, j] * x[i, j] for i in 1:n_workers2 for j in 1:n_tasks2))


3 x[1,1] + 9 x[1,2] + 2 x[1,3] + 2 x[1,4] + 7 x[1,5] + 6 x[2,1] + x[2,2] + 5 x[2,3] + 6 x[2,4] + 6 x[2,5] + 9 x[3,1] + 4 x[3,2] + 7 x[3,3] + 10 x[3,4] + 3 x[3,5] + 2 x[4,1] + 5 x[4,2] + 4 x[4,3] + 2 x[4,4] + x[4,5] + 9 x[5,1] + 6 x[5,2] + 2 x[5,3] + 4 x[5,4] + 6 x[5,5]

In [13]:
# Restricción: cada trabajador hace una tarea
@constraint(model2, [i in 1:n_workers2], sum(x[i, j] for j in 1:n_tasks2) == 1)

# Restricción: cada tarea la hace solo un trabajador
@constraint(model2, [j in 1:n_tasks2], sum(x[i, j] for i in 1:n_workers2) == 1)


5-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 x[1,1] + x[2,1] + x[3,1] + x[4,1] + x[5,1] = 1
 x[1,2] + x[2,2] + x[3,2] + x[4,2] + x[5,2] = 1
 x[1,3] + x[2,3] + x[3,3] + x[4,3] + x[5,3] = 1
 x[1,4] + x[2,4] + x[3,4] + x[4,4] + x[5,4] = 1
 x[1,5] + x[2,5] + x[3,5] + x[4,5] + x[5,5] = 1

In [14]:
# Resolver el modelo
optimize!(model2)

# Mostrar resultados
println("Estado del modelo: ", termination_status(model2))
println("Costo total: ", objective_value(model2))

# Mostrar la asignación de cada trabajador
for i in 1:n_workers2, j in 1:n_tasks2
    if value(x[i, j]) > 0.5
        println("Trabajador ", i, " hace la tarea ", j)
    end
end


Estado del modelo: OPTIMAL
Costo total: 10.0
Trabajador 1 hace la tarea 4
Trabajador 2 hace la tarea 2
Trabajador 3 hace la tarea 5
Trabajador 4 hace la tarea 1
Trabajador 5 hace la tarea 3
